In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Fetch the data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
rows = table.text.split('\n\n\n')

data = list()
for r in rows:
    items = r.replace('\n\n', '').split('\n')
    data.append(items)

df = pd.DataFrame(data=data[1:], columns=data[0])

#### Clean the DataFrame

In [3]:
df = df[df['Borough'] != 'Not assigned']

mask = (df['Neighbourhood'] == 'Not assigned')
df['Neighbourhood'][mask] = df['Borough']

transformed_df = df.groupby(['Postcode'])['Neighbourhood'].apply(','.join).reset_index()
df = pd.merge(left=transformed_df, right=df[['Postcode', 'Borough']], on='Postcode', how='left')
df = df[['Postcode', 'Borough', 'Neighbourhood']]
df.drop_duplicates(inplace=True)

#### Print the shape

In [4]:
print(df.shape)

(103, 3)
